# Routing Topology: OSPF using FRRouting with Chameleon

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Configure the Environment


In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)

from chameleon_utils.chameleon_config import *

load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")

from my_experiment.frrouting_experiment import FRRouting_Experiment


## Create the Experiment


In [2]:
frr_experiment = FRRouting_Experiment('FRRouting_OSPF_SC22')

## (Optional): Query for Available Tesbed Resources and Settings

In [3]:
try:
    frr_experiment.list_sites()
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
MASS,"100 Bigelow Street,Holyoke MA 01040","(42.202493000000004, -72.60787662257826)",3,6,190,192,2,1528,1536,8,60590,60600,10,381,381,0,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
SALT,"572 Delong St,Salt Lake City, UT 84104","(40.75707505789612, -111.95346637770317)",3,6,62,192,130,1144,1536,392,59722,60600,878,355,381,26,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
UTAH,"875 South West Temple,Salt Lake City, UT 84101","(40.75036663265306, -111.893838)",5,10,172,320,148,2160,2560,400,115864,116400,536,621,635,14,2,2,0,3,4,1,16,16,0,4,4,0,5,5,0
WASH,"The Bexley, 1761 Old Meadow Road, McLean, VA 22102, United States of America","(38.91930235, -77.21183383681088)",3,6,90,192,102,968,1536,568,59660,60600,940,355,381,26,2,2,0,1,2,1,10,10,0,2,2,0,3,3,0
DALL,"1950 N Stemmons Fwy,Dallas, TX 75207","(32.800965950000005, -96.81952300449768)",3,6,94,192,98,1210,1536,326,60332,60600,268,368,381,13,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
STAR,,"(0, 0)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UCSD,"10100 Hopkins Dr, San Diego, CA 92121","(32.8881832, -117.2388161)",5,10,280,320,40,2464,2560,96,116210,116400,190,623,635,12,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0
CLEM,"340 Computer Court,Anderson, SC 29625","(34.586543500000005, -82.82128891709674)",3,6,124,192,68,1010,1536,526,60490,60600,110,380,381,1,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
MICH,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",3,6,76,192,116,1072,1536,464,60320,60600,280,374,381,7,2,2,0,1,2,1,10,10,0,2,2,0,3,3,0
FIU,"11001 SW 14th St,Miami, FL 33199","(25.754495891386522, -80.37232833001887)",5,10,176,320,144,2048,2560,512,115982,116400,418,627,635,8,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0


## Define Experiment

The following creates private layer 2 networks on three sites including a OSPF gateway routers that propogate routes acrross the topology. 


In [4]:
frr_sites = { 'dall': { 'site': 'DALL', 'facility': 'FABRIC', 'node_count': 2 },
              'salt': { 'site': 'SALT', 'facility': 'FABRIC', 'node_count': 2 },
              #'cham': { 'site': 'STAR', 'facility': 'CHI@UC', 'node_count': 2 },
              'mich': { 'site': 'MICH', 'facility': 'FABRIC', 'node_count': 2 },
              'utah': { 'site': 'UTAH', 'facility': 'FABRIC', 'node_count': 2 }
            }
    
frr_links = { 'Link12': ('dall','salt'),
              'Link23': ('salt','mich'),
              'Link31': ('mich','dall'),
              'Link24': ('utah','salt')
            }

In [5]:
frr_experiment.configure(frr_sites, frr_links)

In [6]:
frr_experiment.deploy()

Submit FABRIC Slice...


Exception: Failed to submit slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Mon, 07 Nov 2022 14:12:50 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '214', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, PATCH, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': 'Slice FRRouting_OSPF_SC22 already exists'})
HTTP response body: b'{\n    "errors": [\n        {\n            "details": "Slice FRRouting_OSPF_SC22 already exists",\n            "message": "Internal Server Error"\n        }\n    ],\n    "size": 1,\n    "status": 500,\n    "type": "error"\n}'


## Run the GUI and Interact with the Experiment

In [ ]:
frr_experiment.load()
print(f"here")
frr_experiment.display()


## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    frr_experiment.fablib.delete_all()
    #slice.delete()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
frr_experiment.execute_on_all_edge_nodes("rm *output", verbose=True)

In [ ]:
frr_experiment.execute_on_all_edge_nodes("ls", verbose=True)


In [ ]:
frr_sites = { 'dall': { 'site': 'DALL', 'facility': 'FABRIC', 'node_count': 2 },
              'utah': { 'site': 'UTAH', 'facility': 'FABRIC', 'node_count': 2 },

              'salt': { 'site': 'SALT', 'facility': 'FABRIC', 'node_count': 2 },
              'wash': { 'site': 'WASH', 'facility': 'FABRIC', 'node_count': 2 },
              'tacc': { 'site': 'TACC', 'facility': 'FABRIC', 'node_count': 2 },
              'max':  { 'site': 'MAX', 'facility': 'FABRIC', 'node_count': 2 },
              'ncsa': { 'site': 'NCSA', 'facility': 'FABRIC', 'node_count': 2 },
              #'cham': { 'site': 'STAR', 'facility': 'CHI@UC', 'node_count': 2 },
              'cham': { 'site': 'STAR', 'facility': 'FABRIC', 'node_count': 2 },

            }
    
frr_links = { 'Link_dall_tacc': ('dall','tacc'),
              'Link_cham_ncsa': ('cham','ncsa'),
              'Link_salt_utah': ('salt','utah'),
              'Link_wash_max':  ('wash','max'),
              'Link_utah_dall': ('utah','dall'),
              'Link_dall_wash': ('dall','wash'),
              'Link_wash_cham': ('wash','cham'),
              'Link_cham_utah': ('cham','utah'),
              'Link_cham_dall': ('cham','dall'), 
            }

In [ ]:
frr_experiment.wait_jupyter(timeout=20000)

In [ ]:
frr_experiment.execute_on_all_edge_nodes(f'rm -rf fabric_node_tools')
print('deploy3')
frr_experiment.upload_directory_to_all_edge_nodes('fabric_node_tools','.')
print('deploy4')
frr_experiment.execute_on_all_edge_nodes(f'chmod +x fabric_node_tools/*.sh')
print('deploy5')
frr_experiment.configure_devs()
print('deploy6')

In [ ]:
frr_experiment.configure_routers()

In [ ]:
print('deploy7')
command = f'sudo yum install -y -q iperf3 iproute-tc && sudo ./fabric_node_tools/host_tune_redhat.sh'
frr_experiment.execute_on_all_edge_nodes(f'{command}')
print('deploy8')